# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../WeatherPy/Output/Weather Data.csv")

df

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,Fayetteville,35.0527,-78.8784,58.48,89,1,3.44,US,-14400
1,Rikitea,-23.1203,-134.9692,73.83,85,18,21.50,PF,-32400
2,Tasiilaq,65.6145,-37.6368,30.36,93,100,7.02,GL,-7200
3,Seymchan,62.8833,152.4333,2.48,80,97,2.10,RU,39600
4,Nanortalik,60.1432,-45.2371,36.88,97,100,11.32,GL,-7200
...,...,...,...,...,...,...,...,...,...
540,Trat,12.5000,102.5000,87.26,88,100,5.32,TH,25200
541,Tocopilla,-22.0920,-70.1979,59.38,67,15,6.02,CL,-10800
542,Pierre,44.3683,-100.3510,36.86,87,1,1.01,US,-18000
543,Khvoynaya,58.9000,34.5333,42.17,74,56,9.60,RU,10800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
coordinates=[]

for index, row in df.iterrows():
    coordinates.append((row['Latitude'],row['Longitude']))

# coordinates

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

In [6]:
# Assign the marker layer to a variable
markers = gmaps.heatmap_layer(coordinates, weights=df['Humidity'],point_radius=15)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
tempmin = 60
tempmax = 80

df_temp_filt = df.loc[(df["Max Temperature"]<tempmax)&(df["Max Temperature"]<tempmin)]
df_temp_filt

hummax = 50
hummin = 30

df_hum_filt = df_temp_filt.loc[(df_temp_filt["Humidity"]<hummax)&(df_temp_filt["Humidity"]>hummin)]
df_hum_filt

windspeedmax = 10

df_windspeed_filt = df_hum_filt.loc[df_hum_filt['Wind Speed']<windspeedmax]
df_windspeed_filt

Cloudinessmax = 80

df_final = df_windspeed_filt.loc[df_windspeed_filt['Cloudiness']<Cloudinessmax]
df_final

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
209,Laas,46.6166,10.7002,51.03,33,2,1.01,IT,7200
241,Belyy Yar,53.6039,91.3903,44.49,42,0,8.95,RU,25200
365,Tucumcari,35.1717,-103.7250,55.24,35,1,5.75,US,-21600
382,Darhan,49.4867,105.9228,44.55,40,6,1.61,MN,28800
445,Shitanjing,39.2342,106.3439,54.12,37,6,4.03,CN,28800
472,Jinchang,38.4953,102.1739,55.60,42,0,7.43,CN,28800
532,Kurchum,48.5722,83.6542,45.00,38,6,8.37,KZ,21600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = df_final
# hotel_df["Hotel Name"]=""

# Dependencies
import requests
import json
import pprint
hotel_df

,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
209,Laas,46.6166,10.7002,51.03,33,2,1.01,IT,7200
241,Belyy Yar,53.6039,91.3903,44.49,42,0,8.95,RU,25200
365,Tucumcari,35.1717,-103.7250,55.24,35,1,5.75,US,-21600
382,Darhan,49.4867,105.9228,44.55,40,6,1.61,MN,28800
445,Shitanjing,39.2342,106.3439,54.12,37,6,4.03,CN,28800
472,Jinchang,38.4953,102.1739,55.60,42,0,7.43,CN,28800
532,Kurchum,48.5722,83.6542,45.00,38,6,8.37,KZ,21600


In [9]:
hotel_list = []
hlat=[]
hlng=[]

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']

    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        # run a request using our params dictionary
        response = requests.get(base_url, params=params).json()
        hotelname = response["results"][0]["name"]
        CityName = row['City Name']
        print(hotelname)
        hotel_list.append(hotelname)
        hlat.append(response["results"][0]["geometry"]["location"]["lat"])
        hlng.append(response["results"][0]["geometry"]["location"]["lng"])

    except IndexError:
        print(f"No Hotel within 5km of this city")
        hotel_list.append("No Hotel within 5km of this city")
        hlat.append("")
        hlng.append("")

Gasthof Hotel Sonne Silandro
No Hotel within 5km of this city
No Hotel within 5km of this city
Buudai Hotel
No Hotel within 5km of this city
Jinchang Hotel
Gostinitsa "Kneda"


In [10]:
# Store Dataframe under a new name and reset index
hotel_reset = hotel_df
hotel_reset = hotel_reset.reset_index()
hotel_reset

,index,City Name,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,209,Laas,46.6166,10.7002,51.03,33,2,1.01,IT,7200
1,241,Belyy Yar,53.6039,91.3903,44.49,42,0,8.95,RU,25200
2,365,Tucumcari,35.1717,-103.7250,55.24,35,1,5.75,US,-21600
3,382,Darhan,49.4867,105.9228,44.55,40,6,1.61,MN,28800
4,445,Shitanjing,39.2342,106.3439,54.12,37,6,4.03,CN,28800
5,472,Jinchang,38.4953,102.1739,55.60,42,0,7.43,CN,28800
6,532,Kurchum,48.5722,83.6542,45.00,38,6,8.37,KZ,21600


In [11]:
# 
for x in range (0,len(hotel_reset['City Name'])):
    hotel_reset.loc[[x],["Hotel Name"]] = hotel_list[x]
    hotel_reset.loc[[x],["Hotel Lat"]] = hlat[x]
    hotel_reset.loc[[x],["Hotel Lng"]] = hlng[x]
hotel_reset=hotel_reset.rename(columns={'Latitude':'Lat','Longitude':'Lng',"City Name":"City"})
hotel_reset

,index,City,Lat,Lng,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Datetime,Hotel Name,Hotel Lat,Hotel Lng
0,209,Laas,46.6166,10.7002,51.03,33,2,1.01,IT,7200,Gasthof Hotel Sonne Silandro,46.631183,10.75867
1,241,Belyy Yar,53.6039,91.3903,44.49,42,0,8.95,RU,25200,No Hotel within 5km of this city,,
2,365,Tucumcari,35.1717,-103.7250,55.24,35,1,5.75,US,-21600,No Hotel within 5km of this city,,
3,382,Darhan,49.4867,105.9228,44.55,40,6,1.61,MN,28800,Buudai Hotel,49.467263,105.956318
4,445,Shitanjing,39.2342,106.3439,54.12,37,6,4.03,CN,28800,No Hotel within 5km of this city,,
5,472,Jinchang,38.4953,102.1739,55.60,42,0,7.43,CN,28800,Jinchang Hotel,38.520166,102.185906
6,532,Kurchum,48.5722,83.6542,45.00,38,6,8.37,KZ,21600,"Gostinitsa ""Kneda""",48.569545,83.659269


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_reset.iterrows()]
locations = hotel_reset[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…